In [ ]:
""" Raman Rabi Oscillations
    Preston Huft, May 2019
    
    Runs into numerical errors and overflows. Check eqs and parameter vals.
"""

In [ ]:
###############################################################################
## Raman pi/2 pulse for different spatial overlap at atom trap site 
## of detuning from the transition resonance
###############################################################################

## TODO:
# - change eqs to two-photon eqs
# - get power seen by an atom at some fixed x=x0 for as a function of position 
#   of a Gaussian beam with 
#   an intensity given by the known input power and expected beam waist
# - use that power to define a Rabi frequency for a given beam position in x
# - solve the two-photon O.B.E.s for a Rabi freq. at various x values, and
#   store the last point of each soln (how far the state rotated during the 
#   pulse)
# - repeat for off-center sites, which have Rabi freq different detunings 
#   (smaller trap depth, so less A.C. stark shift, less resonant with applied 
#   light)

# def derivs(y0,t,d,D,O1,O2):
#     """ Returns RHS of optical bloch eqs for current values at time t"""
#     c_g,c_r,c_e = y0
    
#     # time derivatives of state amplitudes (Mark's notes, eqs 11.3)
#     dcg = 1j*(cc(O1)*c_r-D*c_g)/2
#     dcr = 1j*(d*c_r+O1*c_g+cc(O2)*c_e)/2
#     dce = 1j*(D*c_e + O2*c_r)/2
    
#     return np.array([dcg,dcr,dce])

def derivs(y0,t,D1,D2,O1,O2):
    """ Returns RHS of optical bloch eqs for current values at time t"""
    c_g,c_r,c_e = y0
    
    # time derivatives of state amplitudes (Derived "by hand" and compared 
    # to Mark's notes, eqs 11.3)
#     dcg = 1j*(cc(O1)*c_r*np.exp(-1j*D1*t))/2
#     dcr = 1j*(O1*c_g*np.exp(1j*D1*t)+O1*c_e+cc(O2)*c_e*np.exp(-1j*D2*t))/2
#     dce = 1j*(O2*c_e*np.exp(1j*D2*t))/2

    # straight out of Mark's notes, except e <-> r, because r is
    # our intermediate state, and e is the final state
    D = D1+D2
    d = D1-D2
    
    dcg = 1j*(cc(O1)*c_r-D*c_g)/2
    dcr = 1j*(d*c_r+O1*c_g+cc(O2)*c_e)/2
    dce = 1j*(D*c_e+O2*c_r)/2
    
    return np.array([dcg,dcr,dce])

## experiment parameters and pertinent quantities
P = 2*(3e-6) # [W] beam power per sideband at atoms
w0 = 6e-6 # [m] expected Gaussian beam waist at atoms

## test a single Rabi frequency
O = 0.200 # [MHz] 324623.79048954 # Two-photon Rabi f
D1 = 2*pi*4.5e4  # [MHz]
D2 = D1
D = D1+D2 
d = D1-D2
O1 = sqrt(D*O) # Single-photon Rabi f, for O1 = O2
O2 = O1

# o1DipoleElem = -1.792e-29 # see rb_matrix_elements
# o2DipoleElem = 1.792e-29

# # Rabi frequencies and the position values to be scanned
# r_vals = np.linspace(-10e-6,10e-6,11) # the positions to scan
# O_vals = (2/(c*e0*hbar**2))*Intensity(P,w0,r_vals) \
#         *o1DipoleElem*o2DipoleElem \
#         /(2*D)

## initial conditions - system starts in ground state (|F=2,mF=0>)
c_g = 1 + 0j
c_r = 0 + 0j
c_e = 0 + 0j
y0 = np.array([c_g,c_r,c_e]) # bundle i.c.

fig = plt.figure()
ax = fig.add_subplot(111)
# ax.set_ylim((0,1))
ax.set_title('pi/2 pulse')
# ax.set_xlabel('position [um]')
ax.set_ylabel('Excitation probability')

## run the parameter scan
# prob_e = np.empty(len(r_vals))

## max pulse duration
t_exp = 5 # [us]  ~ pi/2 time we've been using

## Try with solve_ivp
stopwatch = time.time()
# func = lambda t,y: derivs(y,t,d,D,O1,O2)
func = lambda time,y: derivs(y,time,D1,D2,O1,O2)
soln = solve_ivp(func,[0,t_exp],y0,#t_eval=np.linspace(0,t_exp,50),
                method='RK45')
ctime = time.time()-stopwatch
print(f"Comp time = {ctime}[s]")
ypts = [abs(y)**2 for y in soln.y[2]]
ax.plot(soln.t,ypts)

# ax.legend(loc='upper right')
plt.show()